In [70]:
!pip install tensorflow pillow matplotlib  Image --quiet 


In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

train_dir = "E:/project/pet_adoption_matching_algorithm/dataset/animal/dog"
test_dir = "E:/project/pet_adoption_matching_algorithm/dataset/animal/dog"

num_classes = 70
image_size = (224, 224)  # Corrected variable name
batch_size = 32
learning_rate = 0.01
epochs = 10

training_data = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_data = ImageDataGenerator(rescale=1./255)

base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(image_size[0], image_size[1], 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=["accuracy"])

train_generator = training_data.flow_from_directory(
    train_dir,
    target_size=image_size,  # Corrected variable name
    batch_size=batch_size,
    class_mode="categorical"
)

valid_generator = validation_data.flow_from_directory(
    test_dir,
    target_size=image_size,  # Corrected variable name
    batch_size=batch_size,
    class_mode="categorical"
)

# Calculate steps_per_epoch and validation_steps
steps_per_epoch = train_generator.samples // batch_size
validation_steps = valid_generator.samples // batch_size

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=validation_steps
)

model.save("dogclassification.h5")


Found 8456 images belonging to 70 classes.
Found 8456 images belonging to 70 classes.
Epoch 1/10
264/264 [==============================] - 750s 3s/step - loss: 2.1186 - accuracy: 0.7419 - val_loss: 0.6452 - val_accuracy: 0.8299
Epoch 2/10
264/264 [==============================] - 617s 2s/step - loss: 0.7351 - accuracy: 0.8202 - val_loss: 0.5798 - val_accuracy: 0.8486
Epoch 3/10
264/264 [==============================] - 637s 2s/step - loss: 0.6631 - accuracy: 0.8343 - val_loss: 0.5793 - val_accuracy: 0.8625
Epoch 4/10
264/264 [==============================] - 676s 3s/step - loss: 0.6515 - accuracy: 0.8418 - val_loss: 0.5232 - val_accuracy: 0.8582
Epoch 5/10
264/264 [==============================] - 799s 3s/step - loss: 0.6493 - accuracy: 0.8446 - val_loss: 0.4564 - val_accuracy: 0.8806
Epoch 6/10
264/264 [==============================] - 671s 3s/step - loss: 0.5879 - accuracy: 0.8564 - val_loss: 0.5143 - val_accuracy: 0.8726
Epoch 7/10
264/264 [==============================] - 65

c:\Users\haric\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
